## Import Packages

In [1]:
import sys
sys.path.append('../../')

In [2]:
%load_ext autoreload
import h5py
import numpy as np

import matplotlib.pyplot as plt

from src.m3_learning.be.util import print_be_tree
from src.m3_learning.be.processing import convert_amp_phase
from src.m3_learning.util.h5_util import make_dataset, make_group
from src.m3_learning.util.preprocessing import global_scaler
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
from src.m3_learning.util.file_IO import make_folder
from src.m3_learning.be.processing import SHO_fit_to_array
from src.m3_learning.viz.printing import printer
printing = printer(basepath = './figures/')

%matplotlib inline

ModuleNotFoundError: No module named 'm3_learning'

In [ ]:
# Sets path to file
path = r"./"

In [ ]:
print_be_tree(path + "data_file.h5")

### Extract Data

In [ ]:
# Opens the data file
h5_f = h5py.File(path + "data_file.h5", "r+")

# number of samples per SHO fit
num_bins = h5_f["Measurement_000"].attrs["num_bins"]

# number of pixels in the image
num_pix = h5_f["Measurement_000"].attrs["num_pix"]

# number of pixels in x and y dimensions
num_pix_1d = int(np.sqrt(num_pix))

# number of DC voltage steps
voltage_steps = h5_f["Measurement_000"].attrs["num_udvs_steps"]

# Frequency Vector in Hz
frequency_bin = h5_f["Measurement_000"]["Channel_000"]["Bin_Frequencies"][:]

# Resampled frequency vector
wvec_freq = resample(frequency_bin, 80)

# extracting spectroscopic values
spectroscopic_values = h5_f["Measurement_000"]["Channel_000"]["Spectroscopic_Values"]

# # extract the real and imaginary components
real = h5_f["Measurement_000"]["Channel_000"]['complex']['real'][:]
imag = h5_f["Measurement_000"]["Channel_000"]['complex']['imag'][:]

### Resampling

In [ ]:
# resamples the real and imaginary components
real_resample = resample(real.reshape(num_pix, -1, num_bins), 80, axis=2)
imag_resample = resample(imag.reshape(num_pix, -1, num_bins), 80, axis=2)

# resamples the raw data
raw_data_resample = resample(np.array(h5_f["Measurement_000"]["Channel_000"]["Raw_Data"]).reshape(-1, 165), 80, axis=1)

# resamples the amplitude values
amp_resample, phase_resample = convert_amp_phase(raw_data_resample)

# makes a new object in the h5 file to store the resampled data
make_group(h5_f["Measurement_000"]["Channel_000"], 'magn_spec_resample')

# adds the resampled data to the h5 file
make_dataset(h5_f["Measurement_000"]["Channel_000"], 'raw_data_resample', raw_data_resample)
make_dataset(h5_f["Measurement_000"]["Channel_000"]['complex'], 'real_resample', real_resample)
make_dataset(h5_f["Measurement_000"]["Channel_000"]['complex'], 'imag_resample', imag_resample)
make_dataset(h5_f["Measurement_000"]["Channel_000"]['magn_spec_resample'], 'amp_resample', amp_resample)
make_dataset(h5_f["Measurement_000"]["Channel_000"]['magn_spec_resample'], 'phase_resample', phase_resample)

## Visualizing Input Data


### Comparison of original and resampled data


In [ ]:
# makes the folder
make_folder(f"{path}Assets/Figures/")

# generates random pixel position and timestep
pixel = np.random.randint(
    0, h5_f["Measurement_000"]["Channel_000"]['complex']['real'].shape[0])
timestep = np.random.randint(
    0, h5_f["Measurement_000"]["Channel_000"]['complex']['real'].shape[1]) // num_bins

# plot real and imaginary components of resampled data
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# axs[0].set_title('Magnitude/Phase representation')
axs[0].plot(
    frequency_bin,
    h5_f["Measurement_000"]["Channel_000"]['magn_spec']['amp'][pixel].reshape(-1, num_bins)[
        timestep],
    label="amplitude",
    color="b",
)
axs[0].plot(
    wvec_freq,
    h5_f["Measurement_000"]["Channel_000"]['magn_spec_resample']['amp_resample'][:].reshape(
        num_pix, voltage_steps, -1)[pixel, timestep],
    "o",
    label="amplitude resampled",
    color="b",
)
axs[0].set(xlabel="Frequency (Hz)", ylabel="Amplitude (Arb. U.)")
ax2 = axs[0].twinx()
ax2.plot(
    frequency_bin,
    h5_f["Measurement_000"]["Channel_000"]['magn_spec']['phase'][pixel].reshape(
        -1, num_bins)[timestep],
    label="phase",
    color="r",
)
ax2.plot(
    wvec_freq,
    h5_f["Measurement_000"]["Channel_000"]['magn_spec_resample']['phase_resample'][:].reshape(
        num_pix, voltage_steps, -1)[pixel, timestep],
    "o",
    label="phase resampled",
    color="r",
)
ax2.set(xlabel="Frequency (Hz)", ylabel="Phase (rad)")

# axs[1].set_title('Real and Imaginary')
axs[1].plot(wvec_freq, real_resample[pixel, timestep],
            "s", color='b', label="Real Resample")

axs[1].plot(frequency_bin, h5_f["Measurement_000"]["Channel_000"]['complex']
            ['real'][pixel].reshape(-1, num_bins)[timestep], label="Real", color='b')

axs[1].plot(wvec_freq, h5_f["Measurement_000"]["Channel_000"]['complex']
            ['imag_resample'][pixel, timestep], "s", color='r', label="Imaginary Resample")

axs[1].plot(
    frequency_bin, h5_f["Measurement_000"]["Channel_000"]['complex']['imag'][pixel].reshape(-1, num_bins)[timestep], label="Imaginary", color='r',
)

axs[1].set(xlabel="Frequency (Hz)", ylabel="Amplitude (Arb. U.)")

fig.legend(bbox_to_anchor=(1.16, 0.93), loc="upper right", borderaxespad=0.0)
fig.tight_layout()

printing.savefig(fig, "Figure_4_raw_and_resampled_raw_data.png")


## Data Scaling

In [ ]:
# scale the fit results with Standard Scaler
fit_results_scaler = StandardScaler()
SHO_fit_results = SHO_fit_to_array(h5_f["Measurement_000"]["Channel_000"]["Raw_Data-SHO_Fit_000"]["Fit"])
scaled_fit_results = fit_results_scaler.fit_transform(SHO_fit_results.reshape(-1, 5)[:, 0:4])



In [ ]:
# Loads SHO Fit Results
SHO_fit_results = SHO_fit_to_array(h5_f["Measurement_000"]["Channel_000"]["Raw_Data-SHO_Fit_000"]["Fit"])

# check distributions of each parameter before and after scaling
fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(13, 26))

units = [
    "Amplitude (Arb. U.)",
    "Frequency (Hz)",
    "Quality Factor (Arb. U.)",
    "Phase (rad)",
]

for i in range(4):
    axs[i, 0].hist(SHO_fit_results[:, :, i].flatten(), 100)
    axs[i, 1].hist(scaled_fit_results[:, i].flatten(), 100)

i = 0
for ax in axs.flat:
    ax.set(xlabel=units[i // 2], ylabel="Density")
    i += 1
    ax.ticklabel_format(axis="x", style="sci", scilimits=(0, 0))

axs[0, 0].set_title("Before scaling", fontsize=15)
axs[0, 1].set_title("After scaling", fontsize=15)

printing.savefig(fig, "Figure_5_scaled_fitting_parameters_distributions.png")

In [ ]:
# scale the real component of input data
scaler_real = global_scaler()
scaled_data_real = scaler_real.fit_transform(real_resample).reshape(-1, 80)

# scale the imaginary component of input data
scaler_imag = global_scaler()
scaled_data_imag = scaler_imag.fit_transform(imag_resample).reshape(-1, 80)

# saves the data to the h5_file
make_dataset(h5_f["Measurement_000"]["Channel_000"]['complex'], 'scaled_data', np.stack((scaled_data_real, scaled_data_imag), axis=2))

In [ ]:
h5_f.close()